In [ ]:
import json
import requests
import pandas as pd
import pickle
from tqdm import tqdm_notebook as tqdm
from math import radians, cos, sin, sqrt, atan2
from IPython.display import display, HTML

------

In [ ]:
columns = [
    'agency_id', 
    'service_date_id', 'service_date_date',
    'route_id', 'route_short_name', 'route_long_name',
    'trip_id', 'trip_headsign', 'trip_short_name',
    'stop_time_id', 'stop_time_arrival_time', 'stop_time_departure_time', 'stop_time_stop_sequence', 
    'stop_id', 'stop_stop_id', 'stop_name', 
    'capacity_path_id', 'capacity_path_path', 
    'capacity_capacity_id', 'capacity_capacity_capacity1st', 'capacity_capacity_capacity2nd'
]

In [ ]:
in_dir = "in_data/"
out_dir = "out_data/"

--------

In [ ]:
R = 6373.0
def compute_distance(lat1, lon1, lat2, lon2):
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return (R * c)*1000

max_depth = 10
def bfs(graph, start, end, max_depth=max_depth):
    queue = []
    queue.append([start])
    while queue:
        path = queue.pop(0)
        if len(path) > max_depth:
            return []
        
        node = path[-1]
        
        if node == end:
            return path

        for adjacent in graph.get(node, []):
            new_path = list(path)
            new_path.append(adjacent)
            queue.append(new_path)
            
    return [-1]

-------

### Preparation

In [ ]:
# Dictionary edge_id to feature based on geojson
with open(in_dir + 'edges.geojson') as file:
    edgeid2feature = {}
    data1 = json.load(file)
    for feature in data1['features']:
        edgeid2feature[feature['properties']['edge_id']] = feature

# Dictionary stop_id to feature    
with open(in_dir + 'stations.geojson') as file:
    stopid2coord = {}
    data2 = json.load(file)
    for feature in data2['features']:
        stopid2coord[feature['properties']['station_id']] = feature

        
# Dictionary edge_id to edge_id
# It used to know the neighboring edges of any edges present 
# in the geojson, we allow for 2 meters of imprecision       

max_d = 5
edgeid2edgeid = {}

for edge_id1, feature1 in tqdm(edgeid2feature.items()):
    edge_start_lat1 = feature1['geometry']['coordinates'][0][1]
    edge_start_lng1 = feature1['geometry']['coordinates'][0][0]
    edge_end_lat1 = feature1['geometry']['coordinates'][-1][1]
    edge_end_lng1 = feature1['geometry']['coordinates'][-1][0]
    
    edges = []
    
    for edge_id2, feature2 in edgeid2feature.items():
        if edge_id2 != edge_id1:
            edge_start_lat2 = feature2['geometry']['coordinates'][0][1]
            edge_start_lng2 = feature2['geometry']['coordinates'][0][0]
            edge_end_lat2 = feature2['geometry']['coordinates'][-1][1]
            edge_end_lng2 = feature2['geometry']['coordinates'][-1][0]

            d1 = compute_distance(edge_start_lat1, edge_start_lng1, edge_start_lat2, edge_start_lng2)
            d2 = compute_distance(edge_start_lat1, edge_start_lng1, edge_end_lat2, edge_end_lng2) 
            d3 = compute_distance(edge_end_lat1, edge_end_lng1, edge_start_lat2, edge_start_lng2)
            d4 = compute_distance(edge_end_lat1, edge_end_lng1, edge_end_lat2, edge_end_lng2) 

            if d1 < max_d or d2 < max_d or d3 < max_d or d4 < max_d: 
                if (min(d1,d2,d3,d4) > 0.5):
                    print(edge_id1, min(d1,d2,d3,d4))
                edges.append(edge_id2)
                
    if not edges:
         print(edge_id1)
    
    edgeid2edgeid[edge_id1] = edges    
    
# Dictionary stop_id to edge_id and its reverse
# It is used to know the neighboring edges of any station present in geojson

stopid2edgeid = {}
for stop_id, feature in tqdm(stopid2coord.items()):
    stop_lat = feature['geometry']['coordinates'][1]
    stop_lng = feature['geometry']['coordinates'][0]
    
    edges = []
    
    for edge_id, feature in edgeid2feature.items():
        edge_start_lat = feature['geometry']['coordinates'][0][1]
        edge_start_lng = feature['geometry']['coordinates'][0][0]
        edge_end_lat = feature['geometry']['coordinates'][-1][1]
        edge_end_lng = feature['geometry']['coordinates'][-1][0]
        
        d_start = compute_distance(stop_lat, stop_lng, edge_start_lat, edge_start_lng)
        d_end = compute_distance(stop_lat, stop_lng, edge_end_lat, edge_end_lng)
        if d_start < max_d or d_end < max_d: 
            if (min(d_start,d_end) > 0.5):
                    print(edge_id1, min(d_start,d_end))
            edges.append(edge_id)
            
    if not edges:
        print("Error", stop_id)
    
    stopid2edgeid[stop_id] =  edges

edgeid2stopid = {}
for stopid, edgeids in tqdm(stopid2edgeid.items()):
    for edgeid in edgeids:
        if edgeid in edgeid2stopid:
            edgeid2stopid[edgeid].append(stopid)
        else:
            edgeid2stopid[edgeid] = [stopid]

### Processing

In [ ]:
dates = ['2017-01-30','2017-01-31','2017-02-01','2017-02-02','2017-02-03','2017-02-04','2017-02-05']

In [ ]:
df = pd.concat([pd.read_csv(out_dir + date + '_processed.csv', index_col=0)  for date in dates])
df.columns = columns
grouped = df.groupby(['trip_id', ])

### Prepare keys

In [ ]:
keys = set()

for name, group in tqdm(grouped, desc="Trips"):
    trip = group.sort_values(['stop_time_stop_sequence'])
    
    rows = trip.iterrows()
    last_index, last_stop = next(rows)
    
    for next_index, next_stop in rows:
        stop_1 = str(last_stop.stop_stop_id)
        stop_2 = str(next_stop.stop_stop_id)
        
        if (stop_1, stop_2) not in keys and (stop_2, stop_1) not in keys:
            keys.add((stop_1, stop_2))
    
        last_index, last_stop = (next_index, next_stop)
    
print(len(keys))

### Find all paires of stations and their path

In [ ]:
trips_by_station_id = {}

for key in tqdm(keys):
    stop_1 = key[0] 
    stop_2 = key[1]
        
    if key not in trips_by_station_id and stop_1 in stopid2edgeid and stop_2 in stopid2edgeid:
        start = sorted(stopid2edgeid[stop_1])
        end = sorted(stopid2edgeid[stop_2])

        for s in start:
            for e in end:
                r = bfs(edgeid2edgeid, s, e)
                if key not in trips_by_station_id or len(trips_by_station_id[key]) > len(r):
                    trips_by_station_id[key] = r           
    else:
        print(stop_1, stop_2)

pickle.dump(trips_by_station_id, file=open(out_dir + "trips_by_station_id.dump", 'wb'), protocol=2)

In [ ]:
paths = {}
for name, group in tqdm(grouped.get_group(grouped.get_ke), desc="Trips"):
    trip = group.sort_values(['stop_time_stop_sequence'])
    
    rows = trip.iterrows()
    last_index, last_stop = next(rows)
    
    for next_index, next_stop in rows:
        stop_1 = str(last_stop.stop_stop_id)
        stop_2 = str(next_stop.stop_stop_id)
        
        key1 = (stop_1, stop_2)
        key2 = (stop_2, stop_1)
        
        key_full = (name, last_stop.stop_id)
        
        path = None   
        if key1 in trips_by_station_id:
            path = trips_by_station_id[key1]
        elif key2 in trips_by_station_id:
            path = trips_by_station_id[key2].reverse()
        
        if (key_full not in paths) or (path and len(paths[key_full]) > len(path)):
            paths[key_full] = path
    
        last_index, last_stop = (next_index, next_stop)
        
pickle.dump(paths, file=open(out_dir + "paths.dump", 'wb'), protocol=2)